In [2]:
%load_ext autoreload
%autoreload 2

import logging
import os
import ta
import numpy as np
import pandas as pd

wd = os.path.abspath("__file__").replace("/__file__", "").replace("notebooks", "")
os.chdir(wd)

from datetime import datetime, timedelta, date
from logging.handlers import TimedRotatingFileHandler
from src.utils import get_jinja_yaml_conf, create_db_engine, Clickhouse_client, Postgres_connect
from tqdm.auto import tqdm

now = datetime.now()

In [3]:
conf = get_jinja_yaml_conf('./conf/logging.yml', './conf/data.yml')
tqdm.pandas()

# logger 설정
stream = logging.StreamHandler()
# stream.setLevel(logging.DEBUG)
logger = logging.getLogger('main')
logging.basicConfig(level=eval(conf['logging']['level']),
    format=conf['logging']['format'],
    handlers = [TimedRotatingFileHandler(filename =  conf['logging']['file_name'],
                                when=conf['logging']['when'],
                                interval=conf['logging']['interval'],
                                backupCount=conf['logging']['backupCount']), 
                                   stream]
                )

In [4]:
# Only for notebooks
import re

os.environ['_ts'] = datetime.astimezone(datetime.now()).strftime('%Y-%m-%d %H:%M:%S %z')

with open('./conf/credentials', "r") as file:
    # 각 라인 읽기
    for line in file:
        # 주석(#) 또는 빈 줄은 무시
        if line.strip() == '' or line.startswith('#'):
            continue

        # 각 라인을 '='를 기준으로 key와 value로 분리
        key, value = line.strip().split('=', 1)

        # $ENV 형식의 환경변수가 있을 경우 해당 값을 가져와서 설정
        env_var_pattern = re.compile(r'\$(\w+)')
        matches = env_var_pattern.findall(value)
        for match in matches:
            value = value.replace(f"${match}", os.environ.get(match, "")).replace('"', '')

        # 환경변수로 설정
        os.environ[key] = value

os.environ['full_save'] = 'false'

In [5]:
# DB 설정
engine = create_db_engine(os.environ)
postgres_conn = Postgres_connect(engine)
click_conn = Clickhouse_client(user_name = os.environ['CLICK_USER'], password = os.environ['CLICK_PW'])
full_save = True if click_conn.get_count('stocks', 'daily_trade') == 0 else os.environ['full_save'].lower() == 'true'

2024-08-21 15:14:34,102 (utils.py 60) INFO ::: Connect to 172.20.10.3. DB_NAME is stocks
2024-08-21 15:14:34,108 (utils.py 396) INFO ::: sql execute: SELECT COUNT(*) FROM stocks.daily_trade FINAL


In [25]:
data = postgres_conn.get_data('dart', 'fn_sgl_acct_all')

In [61]:
group_data = data.groupby(['_ts', '고유번호', '접수번호', '사업연도', '보고서코드', '개별연결구분', '재무제표명', '계정id', '계정명', '계정상세']).count()
s = group_data[group_data['정렬순서'] > 1]
s

재무제표구분  \
_ts                              고유번호     접수번호           사업연도 보고서코드 개별연결구분 재무제표명 계정id         계정명      계정상세           
2024-03-12 00:28:49.737241+00:00 00100258 20160518000265 2015 11011 OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -          2   
                                          20160530000811 2016 11013 CFS    재무상태표 -표준계정코드 미사용- 대손충당금    -          2   
                                                                    OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -          2   
                                          20160826000293 2016 11012 CFS    재무상태표 -표준계정코드 미사용- 대손충당금    -          2   
                                                                    OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -          2   
...                                                                                                             ...   
2024-06-16 02:20:22.149634+00:00 00400060 20240613000361 2024 11011 OFS    재무상태표 -표준계정코드 미사용- 계약부채     -          2   
                                                                                              금융리스채권   -          2   
                                                                                              리스부채     -          2   
2024-06-19 21:00:07.155465+00:00 00136101 20240619000040 2024 11011 CFS    현금흐름표 -표준계정코드 미사용- 소송충당부채전입 -          2   
2024-06-26 21:00:07.071767+00:00 00938688 20240626000045 2024 11011 CFS    재무상태표 -표준계정코드 미사용- 팩토링채권    -          2   

                                                                                                             당기명  \
_ts                              고유번호     접수번호           사업연도 보고서코드 개별연결구분 재무제표명 계정id         계정명      계정상세        
2024-03-12 00:28:49.737241+00:00 00100258 20160518000265 2015 11011 OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -       2   
                                          20160530000811 2016 11013 CFS    재무상태표 -표준계정코드 미사용- 대손충당금    -       2   
                                                                    OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -       2   
                                          20160826000293 2016 11012 CFS    재무상태표 -표준계정코드 미사용- 대손충당금    -       2   
                                                                    OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -       2   
...                                                                                                          ...   
2024-06-16 02:20:22.149634+00:00 00400060 20240613000361 2024 11011 OFS    재무상태표 -표준계정코드 미사용- 계약부채     -       2   
                                                                                              금융리스채권   -       2   
                                                                                              리스부채     -       2   
2024-06-19 21:00:07.155465+00:00 00136101 20240619000040 2024 11011 CFS    현금흐름표 -표준계정코드 미사용- 소송충당부채전입 -       2   
2024-06-26 21:00:07.071767+00:00 00938688 20240626000045 2024 11011 CFS    재무상태표 -표준계정코드 미사용- 팩토링채권    -       2   

                                                                                                             당기금액  \
_ts                              고유번호     접수번호           사업연도 보고서코드 개별연결구분 재무제표명 계정id         계정명      계정상세         
2024-03-12 00:28:49.737241+00:00 00100258 20160518000265 2015 11011 OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -        1   
                                          20160530000811 2016 11013 CFS    재무상태표 -표준계정코드 미사용- 대손충당금    -        2   
                                                                    OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -        2   
                                          20160826000293 2016 11012 CFS    재무상태표 -표준계정코드 미사용- 대손충당금    -        2   
                                                                    OFS    재무상태표 -표준계정코드 미사용- 대손충당금    -        2   
...                                                                                                           ...   
2024-06-16 02:20:22.149634+00:00 00400060 20240613000361 2024 11011 OFS    재무상태표 -표준계정코드 미사용- 계약부채     -        2   

In [64]:
s.index[0][1]

'00100258'

In [65]:
p = data[data['고유번호'] == s.index[0][1]]
p = p[p['접수번호'] == s.index[0][2]]
p = p[p['사업연도'] == s.index[0][3]]
p = p[p['보고서코드'] == s.index[0][4]]
p = p[p['개별연결구분'] == s.index[0][5]]
p = p[p['재무제표명'] == s.index[0][6]]
p = p[p['계정id'] == s.index[0][7]]
p = p[p['계정명'] == s.index[0][8]]
p = p[p['계정상세'] == s.index[0][9]]

p

,_ts,사업연도,보고서코드,접수번호,고유번호,개별연결구분,재무제표구분,재무제표명,계정id,계정명,...,당기누적금액,전기명,전기금액,전기누적금액,전기분반기명,전기분반기금액,전전기명,전전기금액,정렬순서,통화단위
6138006,2024-03-12 00:28:49.737241+00:00,2015,11011,20160518000265,00100258,OFS,BS,재무상태표,-표준계정코드 미사용-,대손충당금,...,NaN,제 29 기,2391846.0,NaN,None,NaN,제 28 기,2422593.0,8,KRW
6138025,2024-03-12 00:28:49.737241+00:00,2015,11011,20160518000265,00100258,OFS,BS,재무상태표,-표준계정코드 미사용-,대손충당금,...,NaN,제 29 기,265570.0,NaN,None,NaN,제 28 기,42500.0,27,KRW


In [66]:
p.drop_duplicates()

,_ts,사업연도,보고서코드,접수번호,고유번호,개별연결구분,재무제표구분,재무제표명,계정id,계정명,...,당기누적금액,전기명,전기금액,전기누적금액,전기분반기명,전기분반기금액,전전기명,전전기금액,정렬순서,통화단위
6138006,2024-03-12 00:28:49.737241+00:00,2015,11011,20160518000265,00100258,OFS,BS,재무상태표,-표준계정코드 미사용-,대손충당금,...,NaN,제 29 기,2391846.0,NaN,None,NaN,제 28 기,2422593.0,8,KRW
6138025,2024-03-12 00:28:49.737241+00:00,2015,11011,20160518000265,00100258,OFS,BS,재무상태표,-표준계정코드 미사용-,대손충당금,...,NaN,제 29 기,265570.0,NaN,None,NaN,제 28 기,42500.0,27,KRW


In [67]:
p.iloc[:, 4:].drop_duplicates()

,고유번호,개별연결구분,재무제표구분,재무제표명,계정id,계정명,계정상세,당기명,당기금액,당기누적금액,전기명,전기금액,전기누적금액,전기분반기명,전기분반기금액,전전기명,전전기금액,정렬순서,통화단위
6138006,00100258,OFS,BS,재무상태표,-표준계정코드 미사용-,대손충당금,-,제 30 기,19990296.0,NaN,제 29 기,2391846.0,NaN,None,NaN,제 28 기,2422593.0,8,KRW
6138025,00100258,OFS,BS,재무상태표,-표준계정코드 미사용-,대손충당금,-,제 30 기,NaN,NaN,제 29 기,265570.0,NaN,None,NaN,제 28 기,42500.0,27,KRW
